In [1]:
import time
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import numpy as np
import xgboost as xgb

//anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
dfTrain = pd.read_csv('GBM Analysis - Sheet1.csv')
labels = dfTrain['labels'].values

In [3]:
dfTrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 9 columns):
sampleID        34 non-null int64
temperature     34 non-null float64
bondDuration    34 non-null float64
voltagePower    34 non-null float64
bondForce       34 non-null float64
bondMiddle      34 non-null float64
bondedArea      34 non-null float64
bondItself      34 non-null float64
labels          34 non-null int64
dtypes: float64(7), int64(2)
memory usage: 2.5 KB


In [14]:
trainData, validateData = train_test_split(dfTrain, test_size=0.05,
                                           random_state=42)

In [15]:
le = LabelEncoder()
trainLabels = le.fit_transform(trainData['labels'].values)
validateLabels = le.fit_transform(validateData['labels'].values)

dfTrain['labels'] = dfTrain['labels'].astype(float)

drop_var = ['sampleID', 'labels']

trainData = trainData.drop(drop_var, axis=1)
validateData = validateData.drop(drop_var, axis=1)

trainDMatrix = xgb.DMatrix(trainData.as_matrix(),
                           label=trainLabels.astype(int))

validateDMatrix = xgb.DMatrix(validateData.as_matrix(),
                              label=validateLabels.astype(int))

## Training Classifier for parameter tuning

In [16]:
params = {
    'learning_rate'    : 0.038,
    'colsample_bytree' : 0.6,
    'subsample'        : 0.65,
    'max_depth'        : 7,
    'num_class'        : len(np.unique(trainLabels)),
    'seed'             : 0,
    'objective'        : 'multi:softprob',
    'eval_metric'      : 'merror',
    'booster'          : 'gbtree'
}

In [17]:
watchlist = [(trainDMatrix, 'train'), (trainDMatrix, 'eval')]

In [18]:
start_time = time.time()
clf = xgb.train(params, trainDMatrix, 330, evals=watchlist,
                early_stopping_rounds=10, verbose_eval=True)
print('Time taken to classify')
print((time.time() - start_time)/60)

[0]	train-merror:0.75	eval-merror:0.75
Multiple eval metrics have been passed: 'eval-merror' will be used for early stopping.

Will train until eval-merror hasn't improved in 10 rounds.
[1]	train-merror:0.65625	eval-merror:0.65625
[2]	train-merror:0.625	eval-merror:0.625
[3]	train-merror:0.59375	eval-merror:0.59375
[4]	train-merror:0.59375	eval-merror:0.59375
[5]	train-merror:0.5625	eval-merror:0.5625
[6]	train-merror:0.5625	eval-merror:0.5625
[7]	train-merror:0.5	eval-merror:0.5
[8]	train-merror:0.53125	eval-merror:0.53125
[9]	train-merror:0.46875	eval-merror:0.46875
[10]	train-merror:0.53125	eval-merror:0.53125
[11]	train-merror:0.46875	eval-merror:0.46875
[12]	train-merror:0.5	eval-merror:0.5
[13]	train-merror:0.53125	eval-merror:0.53125
[14]	train-merror:0.5	eval-merror:0.5
[15]	train-merror:0.46875	eval-merror:0.46875
[16]	train-merror:0.46875	eval-merror:0.46875
[17]	train-merror:0.40625	eval-merror:0.40625
[18]	train-merror:0.46875	eval-merror:0.46875
[19]	train-merror:0.4375	ev

### Validation testing

In [19]:
validate_label_predictions = clf.predict(validateDMatrix)
print(accuracy_score(validateLabels, validate_label_predictions.argmax(axis = 1)))

0.0
